In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie

In [ ]:
experiments_folder = "experiments"
if !ispath(experiments_folder)
    mkdir(experiments_folder)
end
no_video = true

In [ ]:
num_atoms = 12
num_bonds = num_atoms - 1
fps = 60
animation_fps = 60
animation_scale = 4
pref_dist = 1.
start_L = num_bonds*pref_dist
break_dist = 1.5*pref_dist
max_pull = 1.5*break_dist - pref_dist
max_push = max_pull
min_crack = 0.1*pref_dist
max_crack = start_L + max_pull - (num_bonds-1)*min_crack
pot_min = -1.
diss_coeff = 0.05
time_pull = 2
l2_dissipation = true # if false => Kelvin-Voigt
soft_max_alpha = 5
time_pause = 0.
time_horizon = 5*time_pull + time_pause
animation_width = 800
hidpi_scaling = 2
fontsize = 16
only_video = true
atom_color = :blue
bond_color = :green
animation_atom_size = 6
animation_bond_width = 2

In [ ]:
function lennard_jones(dist_sq)
    # Lennard-jones of squared distances.
    # The factor cbrt(2) assures that the global minimum 
    # is attained at cur_dist == pref_dist
    q = pref_dist^2/(cbrt(2)*dist_sq)
    -4*pot_min*(q^6 - q^3)
#     q = pref_dist^2/(2*dist_sq)
#     -4*pot_min*(q^2 - q)
end

function dirichlet_delta(t)
    if t <= 2*time_pull
        return max_pull*sin(pi/2*t/time_pull)
    elseif t <= 4*time_pull
        return max_push*sin(pi/2*t/time_pull)
    elseif t <= 5*time_pull
        return max_pull*sin(pi/2*t/time_pull)
    else
        return max_pull
    end
end

In [ ]:
function compute_line_minmoves()
    step = 2

    minmove = Model(
        ()->MadNLP.Optimizer(
            print_level=MadNLP.WARN,
            blas_num_threads=2,
            # acceptable_tol=1e-8,
            # max_iter=1000
        )
    )

    prev_L = start_L
    L = start_L + dirichlet_delta((step-1)/fps)
    # uniform distribution of atoms at the start of the simulation
    @NLparameter(
        minmove, 
        prev_delta_y[i=1:num_bonds] == prev_L / num_bonds
    )
    @NLexpression(
        minmove, prev_y[i=1:num_atoms],
        sum(prev_delta_y[j] for j in 1:i-1),
    )

    @NLparameter(minmove, max_elong_sq[i=1:num_bonds] == value(prev_delta_y[i])^2)
    @NLparameter(minmove, broken[i=1:num_bonds] == 0.)

    delta_L = L - prev_L
    # initial guess: uniform expansion of all bonds
    @variable(
        minmove, min_crack <= delta_y[i=1:num_bonds] <= max_crack,
        start=value(prev_delta_y[i]) + delta_L/num_bonds
    )
    @expression(minmove, y[i=1:num_atoms], sum(delta_y[j] for j in 1:i-1))

    # dirichlet boundary condition
    @constraint(minmove, dirichlet_right, y[end] == L)

    # energy
    @expression(minmove, dist_sq[i=1:num_bonds], delta_y[i]^2)
    register(minmove, :W, 1, lennard_jones, autodiff = true)
    @NLexpression(
        minmove, energy_expr,
        sum((1-broken[i])*W(dist_sq[i]) + broken[i]*(
            (
                W(dist_sq[i])*exp(soft_max_alpha*W(dist_sq[i])) +
                W(max_elong_sq[i])*exp(soft_max_alpha*W(max_elong_sq[i]))
            ) / (exp(soft_max_alpha*W(dist_sq[i])) + exp(soft_max_alpha*W(max_elong_sq[i])))
        ) for i in 1:num_bonds)
    )

    # dissipation
    if l2_dissipation
        @NLexpression(
            minmove, dissipation_expr,
            0.5*sum((y[i] - prev_y[i])^2 for i in 2:num_atoms-1)
        )
    else
        @NLexpression(
            minmove, dissipation_expr,
            0.5*sum((delta_y[i] - prev_delta_y[i])^2 for i in 1:num_bonds)
        )
    end

    @NLobjective(minmove, Min, energy_expr + diss_coeff*fps*dissipation_expr)

    optimize!(minmove)
    
    steps = [
        (value.(prev_y), value.(broken) .== 1.)
        (value.(y), value.(broken) .== 1.)
    ]
    
    @showprogress "Computing minmoves..." for step in 3:fps*time_horizon
        # update memory variable
        y_last = steps[end][1]
        delta_y_last = y_last[2:end] - y_last[1:end-1]
        comp_max_elong_sq = delta_y_last.^2
        new_max_elong_sq = max.(value.(max_elong_sq), comp_max_elong_sq)
        set_value.(max_elong_sq, new_max_elong_sq)
    
        # check for broken links
        new_broken = new_max_elong_sq .> break_dist^2
        set_value.(broken, new_broken)

        # new -> old
        # note: delta_y is the variable not y
        set_value.(prev_delta_y, value.(delta_y_last))
        prev_L = L
        L = start_L + dirichlet_delta((step-1)/fps)

        # update rod length
        set_normalized_rhs(dirichlet_right, L)

        # update start value
        delta_L = L - prev_L
        set_start_value.(delta_y, delta_y_last .+ delta_L/num_bonds)

        optimize!(minmove)

        push!(steps, (value.(y), value.(broken) .== 1.))
    end
    
    steps
end

In [ ]:
function setup_line_fig()  
    aspect = 6
    width = animation_width
    height = width/aspect
    max_right = start_L + max_pull
    min_left = 0.
    delta = 0.25
    max_len = max_right - min_left + 2*delta
    ax_height = max_len/aspect
    
    fig = Figure(
        resolution=(animation_scale*width, animation_scale*height),
        fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_left-delta, max_right+delta, -ax_height/2, ax_height/2),
        aspect=aspect,
    )
    
    hidespines!(ax)
    hidedecorations!(ax)
    
    fig, ax
end

function plot_step!(ax, step)
    y = zeros(num_atoms)
    x, broken = step
    
    for i in 1:num_bonds
        if !broken[i]
            lines!(
                ax, [x[i], x[i+1]], [0, 0],
                color=bond_color,
                linewidth=animation_scale*animation_bond_width
            )
        end
    end
    
    scatter!(
        ax, x, y,
        color=atom_color, 
        markersize=animation_scale*animation_atom_size
    )
end

function plot_line_step(step, file_name=nothing)    
    fig, ax = setup_line_fig()
    
    plot_step!(ax, step)
    if file_name != nothing
        save("$experiments_folder/$file_name.pdf", fig)
    end
    
    fig
end

function animate_line_steps(steps, file_name)
    if no_video
        return
    end
    
    if !only_video
        snapshots_folder = "$experiments_folder/$file_name"
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    fig, ax = setup_line_fig()
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", steps; framerate = animation_fps) do (step)
        empty!(ax)
        plot_step!(ax, step)
        if !only_video
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

function plot_line_steps(steps, snapshots, file_name)
    width = 800
    delta_h = 0.5
    delta_v = 0.75
    aspect = (time_horizon + 2*delta_h)/(start_L+max_pull + 2*delta_v)*1.35
    height = width/aspect
    
    fig = Figure(
        resolution=(animation_scale*width, animation_scale*height),
        fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(-delta_h, time_horizon + delta_h, -delta_v, start_L+max_pull + delta_v),
        aspect=aspect,
    )
    
    hidespines!(ax)
    hidedecorations!(ax)
    
#     # plot the traces of each atom
#     for i in 1:num_atoms
#         lines!(
#             [i/fps for i in 1:length(steps)],
#             [step[1][i] for step in steps],
#             linewidth=animation_scale*1.5,
#             color=:gray
#         )
#     end

    # plot snapshots
    for i in snapshots
        x, broken = steps[i]
        t = i/fps
        
        for j in 1:num_bonds
            if broken[j]
                continue
            end
            lines!(
                ax,
                [t, t], [x[j], x[j+1]],
                color=bond_color,
                linewidth=animation_scale*animation_bond_width
            )
        end
        
        scatter!(
            ax,
            t*ones(num_atoms),
            x, 
            color=atom_color,
            markersize=animation_scale*animation_atom_size
        )
    end
    
    save("$experiments_folder/$file_name.pdf", fig)
    
    fig
end

In [ ]:
steps = compute_line_minmoves()
length(steps)

In [ ]:
plot_line_step(steps[end])

In [ ]:
plot_line_steps(steps, [1:60:length(steps)..., length(steps)], "line_l2")

In [ ]:
animate_line_steps(steps, "line_l2")

In [ ]:
num_atoms = 12
num_bonds = num_atoms - 1
fps = 60
animation_fps = 60
animation_scale = 4
pref_dist = 1.
start_L = num_bonds*pref_dist
break_dist = 1.5*pref_dist
max_pull = 1.7*break_dist - pref_dist
max_push = max_pull
min_crack = 0.1*pref_dist
max_crack = start_L + max_pull - (num_bonds-1)*min_crack
pot_min = -1.
diss_coeff = 0.05
time_pull = 1.7
l2_dissipation = false # if false => Kelvin-Voigt
soft_max_alpha = 5
time_pause = 0.
time_horizon = 5*time_pull + time_pause
animation_width = 800
hidpi_scaling = 2
fontsize = 16
only_video = true
atom_color = :blue
bond_color = :green
animation_atom_size = 6
animation_bond_width = 2
experiments_folder = "experiments"

In [ ]:
steps = compute_line_minmoves()
length(steps)

In [ ]:
plot_line_step(steps[end])

In [ ]:
plot_line_steps(steps, [1:51:length(steps)..., length(steps)], "line_kv")

In [ ]:
animate_line_steps(steps, "line_kv")

In [ ]:
function get_edges(verts)
    edges = Matrix{Tuple{Any, Any}}(undef, num_edges, 2)
    edge_id = 1
    # add horizontal edges
    for i in 1:num_verts_hor-1
        for j in 1:num_verts_ver
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i+1,j]
            edge_id += 1
        end
    end
    # right-leaning odd rows
    for i in 1:num_verts_hor
        for j in 1:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i,j+1]
            edge_id += 1
        end
    end
    # right-leaning even rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i+1,j+1]
            edge_id += 1
        end
    end
    # left-leaning odd rows
    for i in 2:num_verts_hor
        for j in 1:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i-1,j+1]
            edge_id += 1
        end
    end
    # left-leaning even rows
    for i in 1:num_verts_hor
        for j in 2:2:num_verts_ver-1
            edges[edge_id, 1] = verts[i,j]
            edges[edge_id, 2] = verts[i,j+1]
            edge_id += 1
        end
    end 
    edges
end

function get_triangles(verts)
    triangs = Matrix{Tuple{Any, Any}}(undef, num_triangs, 3)
    triang_id = 1
    # upward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 1:2:num_verts_ver-1
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j]
            triangs[triang_id, 3] = verts[i,j+1]
            triang_id += 1
        end
    end
    # upward-pointing even rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver-1
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j]
            triangs[triang_id, 3] = verts[i+1,j+1]
            triang_id += 1
        end
    end
    # downward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 3:2:num_verts_ver
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i,j-1]
            triangs[triang_id, 3] = verts[i+1,j]
            triang_id += 1
        end
    end
    # downward-pointing odd rows
    for i in 1:num_verts_hor-1
        for j in 2:2:num_verts_ver
            triangs[triang_id, 1] = verts[i,j]
            triangs[triang_id, 2] = verts[i+1,j-1]
            triangs[triang_id, 3] = verts[i+1,j]
            triang_id += 1
        end
    end
    triangs
end

function vector_to_values(vec)
    broadcast((p) -> value.(p), vec)
end

# currently does not improve that much
function improve_guess(step)
    lowest_objective = objective_value(minmove)
    for crack_start in 1:(num_triangs_hor - num_triangs_ver÷2 - 1)
        for j in 1:num_free_verts_ver
            cracked_edge = crack_start + (j-1)÷2
            for i in 1:num_free_verts_hor
                if i != cracked_edge
                    set_start_value(x[i,j], value(prev_x[i,j]))
                else
                    set_start_value(x[i,j], value(prev_x[i,j]) + delta_width)
                end
            end
        end
        optimize!(minmove)
        comp_lowest_objective = objective_value(minmove)
        if comp_lowest_objective < lowest_objective
#             println("improved at step $step: $lowest_objective -> $comp_lowest_objective")
            steps[end] = (vector_to_values.((vertices, edges, triangles))..., value.(broken) .== 1.)
            lowest_objective = comp_lowest_objective
        end
    end
end

In [ ]:
function plot_square_step!(ax, vertices, edges, triangles, broken)
#     plot_triangles!(ax, triangles)
    plot_edges!(ax, edges, broken)
    plot_vertices!(ax, vertices)
end

function plot_square_step(step, file_name=nothing)
    fig = Figure(
        resolution=(
            animation_scale*animation_width,
            animation_scale*animation_height
        ), fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_x-0.25, max_x+0.25, min_y, max_y),
        aspect=animation_aspect,
    )
    
    hidespines!(ax)
    hidedecorations!(ax)
    
    plot_square_step!(ax, vector_to_values.(step)...)
    
    if file_name != nothing
        save("$experiments_folder/$file_name.pdf", fig)
    end

    fig
end

function plot_vertices!(ax, vertices)
    vertices = vertices[:]
    x = [v[1] for v in vertices]
    y = [v[2] for v in vertices]
    scatter!(ax, x, y, color=atom_color, markersize=animation_scale*animation_atom_size)
end

function plot_edges!(ax, edges, broken)
    num_edges = size(edges)[1]
    for i in 1:num_edges
        if broken[i]
            continue
        end
        v1, v2 = edges[i, :]
        x1, y1 = v1
        x2, y2 = v2
        lines!(
            ax, [x1, x2], [y1, y2],
            color=bond_color,
            linewidth=animation_scale*animation_bond_width,
#             linestyle=:dash
        )
    end
end

function plot_triangles!(ax, triangles)
    num_triags = size(triangles)[1]

    for i in 1:num_triags
        poly!(ax, Tuple.(triangles[i, :]), color=(:pink, 0.25))
    end
end

function animate_square_steps(steps, file_name)
    if no_video
        return
    end
    
    if !only_video
        snapshots_folder = "$experiments_folder/$file_name"
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    fig = Figure(
        resolution=(
            animation_scale*animation_width,
            animation_scale*animation_height
        ), fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_x-0.25, max_x+0.25, min_y, max_y),
        aspect=animation_aspect,
    )
    
    hidespines!(ax)
    hidedecorations!(ax)
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", steps; framerate=animation_fps) do (step)
        empty!(ax)
        plot_step!(ax, step...)
        if !only_video
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

In [ ]:
function compute_square_minmoves()
        minmove = Model(
        ()->MadNLP.Optimizer(
            print_level=MadNLP.WARN,
            blas_num_threads=8,
            # acceptable_tol=1e-8,
            # max_iter=1000
        )
    )

    # minmove = Model(Ipopt.Optimizer)
    # set_optimizer_attribute(minmove, "max_cpu_time", 60.0)
    # set_optimizer_attribute(minmove, "print_level", 0)

    step = 2
    prev_width = start_width
    prev_height = 0
    delta_L = dirichlet_delta((step-1)/fps)
    cur_width = start_width + delta_L*cos(pull_angle)
    cur_height = delta_L*sin(pull_angle)

    @NLparameter(
        minmove,
        prev_x[i=1:num_free_verts_hor,j=1:num_free_verts_ver] ==
            i*triang_side_length + ((j+1) % 2)*triang_side_length/2
    )
    @NLparameter(
        minmove,
        prev_y[i=1:num_free_verts_hor,j=1:num_free_verts_ver] ==
            (j-1)*triang_height
    )

    delta_width = cur_width - prev_width
    delta_height = cur_height - prev_height
    # apply uniform correction
    @variable(
        minmove,
        min_x <= x[i=1:num_free_verts_hor,j=1:num_free_verts_ver] <= max_x,
        start = value(prev_x[i,j]) + delta_width/num_free_verts_hor
    )
    @variable(
        minmove,
        min_y <= y[i=1:num_free_verts_hor,j=1:num_free_verts_ver] <= max_y,
        start = value(prev_y[i,j]) + delta_width/num_free_verts_ver
    )

    # dirichlet condition
    @NLparameter(
        minmove,
        prev_bdry_x[j=1:num_free_verts_ver] == prev_width + ((j+1) % 2)*triang_side_length/2
    )
    @NLparameter(
        minmove,
        prev_bdry_y[j=1:num_free_verts_ver] == prev_height + (j-1)*triang_height
    )
    @NLparameter(
        minmove,
        bdry_x[j=1:num_free_verts_ver] == cur_width + ((j+1) % 2)*triang_side_length/2
    )
    @NLparameter(
        minmove,
        bdry_y[j=1:num_free_verts_ver] == cur_height + (j-1)*triang_height
    )

    # vertices 
    prev_vertices = Matrix{Any}(undef, num_verts_hor, num_verts_ver)
    vertices = Matrix{Any}(undef, num_verts_hor, num_verts_ver)
    # leftmost (fixed)
    for j in 1:num_verts_ver
        prev_vertices[1,j] = (
            ((j+1) % 2)*triang_side_length/2,
            (j-1)*triang_height
        )
        vertices[1,j] = prev_vertices[1,j]
    end
    # middle (free)
    for i in 1:num_free_verts_hor
        for j in 1:num_free_verts_ver
            prev_vertices[i+1,j] = (
                prev_x[i,j],
                prev_y[i,j]
            )
            vertices[i+1,j] = (
                x[i,j],
                y[i,j]
            )
        end
    end
    # rightmost (driven by the boundary condition)
    for j in 1:num_verts_ver
        prev_vertices[num_verts_hor, j] = (
            prev_bdry_x[j],
            prev_bdry_y[j]
        )
        vertices[num_verts_hor, j] = (
            bdry_x[j],
            bdry_y[j]
        )
    end

    prev_edges = get_edges(prev_vertices)
    edges = get_edges(vertices)

    prev_triangles = get_triangles(prev_vertices)
    triangles = get_triangles(vertices)

    if !l2_dissipation
        @NLexpression(minmove, prev_grad11[i=1:num_triangs], prev_triangles[i,2][1]-prev_triangles[i,1][1])
        @NLexpression(minmove, prev_grad21[i=1:num_triangs], prev_triangles[i,2][2]-prev_triangles[i,1][2])
        @NLexpression(
            minmove,
            prev_grad12[i=1:num_triangs],
            1/sqrt(3)*(
                2*(prev_triangles[i,3][1]-prev_triangles[i,1][1])
                - (prev_triangles[i,2][1]-prev_triangles[i,1][1])
            )
        )
        @NLexpression(
            minmove,
            prev_grad22[i=1:num_triangs],
            1/sqrt(3)*(
                2*(prev_triangles[i,3][2]-prev_triangles[i,1][2])
                - (prev_triangles[i,2][2]-prev_triangles[i,1][2])
            )
        )

        @NLexpression(minmove, grad11[i=1:num_triangs], triangles[i,2][1]-triangles[i,1][1])
        @NLexpression(minmove, grad21[i=1:num_triangs], triangles[i,2][2]-triangles[i,1][2])
        @NLexpression(
            minmove,
            grad12[i=1:num_triangs],
            1/sqrt(3)*(
                2*(triangles[i,3][1]-triangles[i,1][1])
                - (triangles[i,2][1]-triangles[i,1][1])
            )
        )
        @NLexpression(
            minmove,
            grad22[i=1:num_triangs],
            1/sqrt(3)*(
                2*(triangles[i,3][2]-triangles[i,1][2])
                - (triangles[i,2][2]-triangles[i,1][2])
            )
        )
    end

    if l2_dissipation
        @NLexpression(
            minmove,
            dissipation,
            sum(
                (x[i,j]-prev_x[i,j])^2 + (y[i,j]-prev_y[i,j])^2
                for i in 1:num_free_verts_hor, j in 1:num_free_verts_ver
            )
        )
    else
        @NLexpression(
            minmove,
            dissipation,
            sum(
                (grad11[i] - prev_grad11[i])^2
                + (grad12[i] - prev_grad12[i])^2
                + (grad21[i] - prev_grad21[i])^2
                + (grad22[i] - prev_grad22[i])^2
                for i in 1:num_triangs
            )
        )
    end

    @expression(minmove, v1[i=1:num_edges], edges[i,1])
    @expression(minmove, v2[i=1:num_edges], edges[i,2])
    @NLexpression(
        minmove,
        dist_sq[i=1:num_edges],
        (v2[i][1] - v1[i][1])^2 + (v2[i][2] - v1[i][2])^2
    )
    # we implicitely assume that the simulation starts
    # in a globally optimal state
    @NLparameter(minmove, max_dist_sq[i=1:num_edges] == pref_dist)
    @NLparameter(minmove, broken[i=1:num_edges] == 0.)

    register(minmove, :W, 1, lennard_jones, autodiff = true)
    # no memory for the moment
    @NLexpression(
        minmove,
        energy,
        sum((1-broken[i])*W(dist_sq[i]) + broken[i]*(
            (
                W(dist_sq[i])*exp(soft_max_alpha*W(dist_sq[i])) +
                W(max_dist_sq[i])*exp(soft_max_alpha*W(max_dist_sq[i]))
            ) / (exp(soft_max_alpha*W(dist_sq[i])) + exp(soft_max_alpha*W(max_dist_sq[i])))
        ) for i in 1:num_edges)
    )

    @NLobjective(
        minmove,
        Min,
        energy + diss_coeff*fps*dissipation
    )

    optimize!(minmove)

    steps = [(vector_to_values.((prev_vertices, prev_edges, prev_triangles))..., value.(broken) .== 1.)]
    push!(steps, (vector_to_values.((vertices, edges, triangles))..., value.(broken) .== 1.));
    
    if try_crack && !l2_dissipation && delta_L > 0
        improve_guess(step)
    end
    
    @showprogress "Computing minmoves..." for step in 3:fps*time_horizon
        # update memory variables
        last_dist_sq = value.(dist_sq)
        new_max_dist_sq = max.(value.(max_dist_sq), last_dist_sq)
        set_value.(max_dist_sq, new_max_dist_sq)

        # check for broken links
        new_broken = new_max_dist_sq .> break_dist^2
        set_value.(broken, new_broken)

        # new -> old
        set_value.(prev_y, value.(y))
        set_value.(prev_x, value.(x))
        prev_width = cur_width
        prev_height = cur_height
        delta_L = dirichlet_delta((step-1)/fps)
        cur_width = start_width + delta_L*cos(pull_angle)
        cur_height = delta_L*sin(pull_angle)
        delta_width = cur_width - prev_width
        delta_height = cur_height - prev_height

        # update right boundary
        set_value.(prev_bdry_x, value.(bdry_x))
        set_value.(prev_bdry_y, value.(bdry_y))
        set_value.(bdry_x, value.(bdry_x) .+ delta_width)
        set_value.(bdry_y, value.(bdry_y) .+ delta_height)

        # update start values
        new_start_x = value.(x) .+ delta_width/num_free_verts_hor
        new_start_y = value.(y) .+ delta_height/num_free_verts_ver
        set_start_value.(x, new_start_x)
        set_start_value.(y, new_start_y)

        optimize!(minmove)
        push!(steps, (vector_to_values.((vertices, edges, triangles))..., value.(broken) .== 1.))

        if try_crack && !l2_dissipation && delta_L > 0
            improve_guess(step)
        end
    end
    
    steps
end

In [ ]:
fps = 60
animation_fps = 60
num_triangs_hor = 12 # choose even
num_triangs_ver = 10 # choose even
num_triangs = 2*num_triangs_hor * num_triangs_ver
num_edges = num_triangs_hor * (num_triangs_ver + 1) + num_triangs_ver * (2*num_triangs_hor + 1)
num_free_verts_hor = num_triangs_hor - 1 # ignore leftmost and rightmost vertices
num_free_verts_ver = num_triangs_ver + 1
num_verts_hor = num_free_verts_hor + 2
num_verts_ver = num_free_verts_ver
triang_side_length = 1
triang_height = sqrt(3)/2 * triang_side_length
pref_dist = triang_side_length
break_dist = 1.5*pref_dist
start_width = num_triangs_hor*triang_side_length
max_pull = 3.6*break_dist - pref_dist
max_pull_2 = max_pull
max_push = pref_dist
pull_angle = 0.
min_x = 0
max_x = start_width + max_pull_2*cos(pull_angle) + triang_side_length/2
min_y = -triang_height + min(0, max_pull_2*sin(pull_angle))
max_y = (num_triangs_ver + 1)*triang_height + max(0, max_pull_2*sin(pull_angle))
diss_coeff = 0.0005
time_pull = 3
l2_dissipation = false # if false => Kelvin-Voigt
time_pause = time_pull/2
time_horizon = 5*time_pull + time_pause
animation_width = 800
animation_aspect = (max_x - min_x) / (max_y - min_y)
animation_height = animation_width / animation_aspect
eps = 0.1
try_crack = false
only_video = true # if false generate a folder of snapshots for each frame

In [ ]:
steps = compute_square_minmoves()
length(steps)

In [ ]:
plot_square_step(steps[end])

In [ ]:
plot_square_step(steps[29], "square_hor_pull_29")

In [ ]:
plot_square_step(steps[30], "square_hor_pull_30")

In [ ]:
plot_square_step(steps[31], "square_hor_pull_31")

In [ ]:
plot_square_step(steps[60], "square_hor_pull_60")

In [ ]:
plot_square_step(steps[61], "square_hor_pull_61")

In [ ]:
plot_square_step(steps[490], "square_hor_pull_490")

In [ ]:
plot_square_step(steps[730], "square_hor_pull_730")

In [ ]:
plot_square_step(steps[end], "square_hor_pull_$(length(steps))")

In [ ]:
animate_square_steps(steps, "square_hor_pull")

In [ ]:
fps = 60
animation_fps = 60
num_triangs_hor = 12 # choose even
num_triangs_ver = 10 # choose even
num_triangs = 2*num_triangs_hor * num_triangs_ver
num_edges = num_triangs_hor * (num_triangs_ver + 1) + num_triangs_ver * (2*num_triangs_hor + 1)
num_free_verts_hor = num_triangs_hor - 1 # ignore leftmost and rightmost vertices
num_free_verts_ver = num_triangs_ver + 1
num_verts_hor = num_free_verts_hor + 2
num_verts_ver = num_free_verts_ver
triang_side_length = 1
triang_height = sqrt(3)/2 * triang_side_length
pref_dist = triang_side_length
break_dist = 1.5*pref_dist
start_width = num_triangs_hor*triang_side_length
max_pull = 3*break_dist - pref_dist
max_pull_2 = max_pull
max_push = 0.8*pref_dist
pull_angle = pi/4
min_x = 0
max_x = start_width + max_pull_2*cos(pull_angle) + triang_side_length/2
min_y = -triang_height + min(0, max_pull_2*sin(pull_angle))
max_y = (num_triangs_ver + 1)*triang_height + max(0, max_pull_2*sin(pull_angle))
diss_coeff = 0.0005
time_pull = 3
l2_dissipation = false # if false => Kelvin-Voigt
time_pause = 0
time_horizon = time_pull
animation_width = 800
animation_aspect = (max_x - min_x) / (max_y - min_y)
animation_height = animation_width / animation_aspect
eps = 0.1
try_crack = false
only_video = true # if false generate a folder of snapshots for each frame

In [ ]:
steps = compute_square_minmoves()
length(steps)

In [ ]:
plot_square_step(steps[end])

In [ ]:
plot_square_step(steps[32], "square_diag_pull_32")

In [ ]:
plot_square_step(steps[33], "square_diag_pull_33")

In [ ]:
plot_square_step(steps[34], "square_diag_pull_34")

In [ ]:
plot_square_step(steps[61], "square_diag_pull_61")

In [ ]:
plot_square_step(steps[62], "square_diag_pull_62")

In [ ]:
plot_square_step(steps[end], "square_hor_pull_$(length(steps))")

In [ ]:
animate_square_steps(steps, "square_diag_pull")

In [ ]:
fps = 60
animation_fps = 60
num_triangs_hor = 12 # choose even
num_triangs_ver = 10 # choose even
num_triangs = 2*num_triangs_hor * num_triangs_ver
num_edges = num_triangs_hor * (num_triangs_ver + 1) + num_triangs_ver * (2*num_triangs_hor + 1)
num_free_verts_hor = num_triangs_hor - 1 # ignore leftmost and rightmost vertices
num_free_verts_ver = num_triangs_ver + 1
num_verts_hor = num_free_verts_hor + 2
num_verts_ver = num_free_verts_ver
triang_side_length = 1
triang_height = sqrt(3)/2 * triang_side_length
pref_dist = triang_side_length
break_dist = 1.5*pref_dist
start_width = num_triangs_hor*triang_side_length
max_pull = 3.5*break_dist - pref_dist
max_pull_2 = max_pull
max_push = 0.8*pref_dist
pull_angle = 0.
min_x = 0
max_x = start_width + max_pull_2*cos(pull_angle) + triang_side_length/2
min_y = -triang_height + min(0, max_pull_2*sin(pull_angle))
max_y = (num_triangs_ver + 1)*triang_height + max(0, max_pull_2*sin(pull_angle))
diss_coeff = 1.
time_pull = 3
l2_dissipation = false # if false => Kelvin-Voigt
time_pause = 0
time_horizon = time_pull
animation_width = 800
animation_aspect = (max_x - min_x) / (max_y - min_y)
animation_height = animation_width / animation_aspect
eps = 0.1
try_crack = false
only_video = true # if false generate a folder of snapshots for each frame

In [ ]:
steps = compute_square_minmoves()
length(steps)

In [ ]:
plot_square_step(steps[end])

In [ ]:
plot_square_step(steps[60], "square_hor_pull_large_nu_60")

In [ ]:
plot_square_step(steps[132], "square_hor_pull_large_nu_132")

In [ ]:
plot_square_step(steps[133], "square_hor_pull_large_nu_133")

In [ ]:
plot_square_step(steps[134], "square_hor_pull_large_nu_134")

In [ ]:
plot_square_step(steps[150], "square_hor_pull_large_nu_150")

In [ ]:
plot_square_step(steps[end], "square_hor_pull_large_nu_$(length(steps))")

In [ ]:
animate_square_steps(steps, "square_hor_pull_large_nu")